In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_classif
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import PCA

In [2]:
seed_value = 50
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [3]:
data = pd.read_csv('diabetes_prediction_dataset.csv')
data.dropna(inplace=True)
data['gender'] = data['gender'].map({'Male': 1, 'Female': 0})
data = pd.get_dummies(data, columns=['smoking_history'], drop_first=True)
data.fillna(0, inplace=True)

In [ ]:
scaler = None
pca = None
def data_preprocessing(X, y, fit):

    # 🔁 Scale tập dữ liệu
    log_transformer = FunctionTransformer(np.log1p, validate=True)

    # Áp dụng vào X_train
    X = log_transformer.transform(X)
    X = pd.DataFrame(X)  # Chuyển lại thành DataFrame nếu cần dùng .fillna()
    X.fillna(0, inplace=True)

    global scaler
    if fit:
        scaler = MinMaxScaler()
        X = scaler.fit_transform(X)
    else:
        X = scaler.transform(X)

    # 🔎 Kiểm tra và xử lý giá trị âm trước khi áp dụng log-transform
    if (X < 0).any():
        print("⚠️ Warning: Negative values detected in X_train. Log transformation might fail.")
    
    # PCA 99%
    global pca
    if fit:
        pca = PCA(n_components=0.99)
        X = pca.fit_transform(X)
        print(f"Số đặc trưng ban đầu: {X.shape[1]}")
        print(f"Số đặc trưng sau PCA (giữ 99% variance): {pca.n_components_}")
        print(f"Tỉ lệ variance giữ lại: {sum(pca.explained_variance_ratio_):.4f}")
    else:
        X = pca.transform(X)

    return X, y

In [9]:
X = data.drop(['diabetes'], axis=1)
y = data['diabetes']
X_processed, y_processed = data_preprocessing(X, y, fit=True)

✅ PCA giữ lại 11 thành phần chính
🔢 Tỉ lệ phương sai giải thích (explained variance ratio):
  PC1: 0.2710
  PC2: 0.2219
  PC3: 0.0972
  PC4: 0.0848
  PC5: 0.0703
  PC6: 0.0540
  PC7: 0.0518
  PC8: 0.0422
  PC9: 0.0414
  PC10: 0.0344
  PC11: 0.0235
📈 Tổng phương sai giữ lại: 0.9924


c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but FunctionTransformer was fitted without feature names
  warnings.warn(
